In [1]:
import pandas as pd
import time

time_start=time.time()
answer_info=pd.read_csv('answer_info.txt',sep='\s+',names=['aid','qid','uid','atime','ainfo','ainfoc','fine',
                                                                 'recommend','table','picture','video','numword',
                                                                 'thumbsup','thumbsdown','comment','collect','thank',
                                                                 'report','nohelp','oppose'])
invite_info=pd.read_csv('invite_info.txt',sep='\s+',names=['qid','uid','time','target'])
invite_info_evaluate=pd.read_csv('invite_info_evaluate_1.txt',sep='\s+',names=['qid','uid','time'])
# member_info=pd.read_csv('member_info.txt',sep='\s+',names=['uid','sex','key_word','num_level','hot_level','regis_type','regis_platform',
#                                                           'look_freq','a','b','c','d','e','A','B','C','D','E','salt','l_topic','topic_n'])
#ques_info=pd.read_csv('question_info.txt',sep='\s+',names=['qid','qtime','qtitle','qtitlec','qinfo','qinfoc','qtopic'])

In [2]:
answer_info['id']=answer_info['qid']+answer_info['uid']
invite_info['id']=invite_info['qid']+invite_info['uid']

In [3]:
answer_info=pd.merge(answer_info,invite_info,how='left',on='id')

In [4]:
x=answer_info[answer_info['target'].isnull()]

In [5]:
len(x)

3013096

In [6]:
x=x.drop_duplicates('aid')

In [7]:
x=x[['aid', 'qid_x', 'uid_x', 'atime', 'ainfo', 'ainfoc']]

In [8]:
x=x[['aid', 'qid_x', 'uid_x', 'atime',]]

In [9]:
x.columns=['aid', 'qid', 'uid', 'atime',]

In [10]:
ques_info=pd.read_csv('question_info.txt',sep='\s+',names=['qid','qtime','qtitle','qtitlec','qinfo','qinfoc','qtopic'])

In [11]:
time_fea=pd.merge(x,ques_info,on='qid',how='left')

In [12]:
def caul_time(x):
    time=x[1:].split('-H')
    return int(time[0])*24+int(time[1])

In [13]:
from tqdm import tqdm
tqdm.pandas()

In [14]:
time_fea['qtime']=time_fea['qtime'].progress_apply(lambda x:caul_time(x))
time_fea['atime']=time_fea['atime'].progress_apply(lambda x:caul_time(x))

100%|██████████| 3012082/3012082 [00:15<00:00, 197034.86it/s]


In [15]:
time_fea['qaStime']=time_fea['atime']-time_fea['qtime']

In [16]:
feature=time_fea[['uid','qaStime']].groupby(['uid']).agg(['mean','max','min'])

In [17]:
feature=feature.reset_index()

In [18]:
feature.columns=['uid','qaStime_mean','qaStime_max','qaStime_min']

In [25]:
y=pd.read_csv('feature_invite_time.csv')

In [20]:
train=pd.read_csv('invite_info.txt',sep='\s+',names=['qid','uid','time','target'])
test=pd.read_csv('invite_info_evaluate_1.txt',sep='\s+',names=['qid','uid','time'])

In [21]:
all_data=pd.concat([train[['qid','uid','time']],test])

In [22]:
all_data=pd.merge(all_data,feature,how='left',on='uid')

In [53]:
len(all_data)

10630845

In [55]:
all_data=pd.concat([all_data,y],axis=1)

In [59]:
all_data=all_data.dropna(subset=['qaStime_mean'])

In [63]:
all_data['itimeSmean']=all_data['invtimeSqtime']-all_data['qaStime_mean']
all_data['itimeSmax']=all_data['invtimeSqtime']-all_data['qaStime_max']
all_data['itimeSmin']=all_data['invtimeSqtime']-all_data['qaStime_min']

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [65]:
all_data[['qid','uid','itimeSmean','itimeSmax','itimeSmin']].to_csv('feature_Stime_history.csv',index=False)

In [66]:
feature.head()

,uid,qaStime_mean,qaStime_max,qaStime_min
0,M1000000983,14908.333333,43744,476
1,M1000020034,12203.000000,12203,12203
2,M1000024720,571.500000,1480,43
3,M1000031027,37277.000000,37277,37277
4,M1000043012,303.222222,5088,0


In [67]:
feature.to_csv('feature_time_history.csv',index=False)